# Budget Execution
#### A Machine Learning Model
___

In [ ]:
#### Required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error, r2_score
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier, LocalOutlierFactor, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (RandomForestClassifier, GradientBoostingClassifier, IsolationForest,
                              HistGradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier)

from sklearn.svm import SVC, SVR, OneClassSVM
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.decomposition import PCA, IncrementalPCA, TruncatedSVD, FactorAnalysis
from sklearn.manifold import Isomap, TSNE
from sklearn.cluster import (KMeans, MiniBatchKMeans, DBSCAN, OPTICS,
                             AgglomerativeClustering, Birch, SpectralClustering)

from sklearn.covariance import EllipticEnvelope
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import (LogisticRegression, RidgeClassifier, SGDClassifier, Perceptron,
                                  LinearRegression, Ridge, Lasso, ElasticNet, BayesianRidge,
                                  HuberRegressor, SGDRegressor)
from sklearn.ensemble import (RandomForestRegressor, GradientBoostingRegressor,
                              HistGradientBoostingRegressor, AdaBoostRegressor)

import statsmodels.api as sm
from sklearn.neural_network import MLPRegressor
from sklearn.inspection import permutation_importance
from xgboost import XGBClassifier
from scipy.stats import zscore, ttest_ind, f_oneway, chi2_contingency, pearsonr, spearmanr
import warnings

hdr = '\r\n' + '-' * 120 + '\r\n'
nwln = '\r\n'
warnings.filterwarnings( 'ignore' )

## I. Abstract

In the evolving landscape of public financial management, data-driven decision-making is increasingly essential. The United States federal budget, comprising trillions of dollars annually, is executed across thousands of accounts managed by hundreds of agencies. Despite decades of modernization, the analytical capacity to predict spending trends, identify execution anomalies, and classify budget behaviors remains underutilized.

This study presents a comprehensive application of machine learning (ML) and statistical modeling techniques to analyze federal account balances. Drawing on a full fiscal year's worth of Treasury Account Symbol (TAS)-level data, we demonstrate how a suite of supervised and unsupervised models can illuminate budgetary trends, detect execution outliers, reduce dimensional complexity, and enhance interpretability.

## II. Methods

### Data Source and Preparation

The dataset used comprises 56,987 records from the Treasury's Federal Account Symbols, with variables including AnnualAppropriations, BorrowingAuthority, ContractAuthority, OffsettingReceipts, Obligations, UnobligatedBalance, and Outlays. All missing values were imputed using SimpleImputer (mean strategy), and features were scaled using StandardScaler.

### Dimensionality Reduction

To facilitate visualization and mitigate multicollinearity, we applied the following techniques:
- Principal Component Analysis (PCA)
- Incremental PCA
- Truncated Singular Value Decomposition (TruncatedSVD)
- Factor Analysis
- Isometric Mapping (Isomap)
- t-distributed Stochastic Neighbor Embedding (t-SNE)

Each method projected the financial feature space into 2D, enabling visual inspection of structure and clustering potential.

### Classification and Clustering

A binary classification task was formulated to identify accounts with above-median obligations. Models included:
- Linear: LogisticRegression, RidgeClassifier, SGDClassifier
- Nonlinear: KNeighbors, DecisionTree, RandomForest, SVC
- Ensemble: AdaBoost, Bagging, GradientBoosting, XGBoost
- Neural Networks: MLPClassifier
- Probabilistic: Naive Bayes, LDA, QDA
- Clustering: KMeans, DBSCAN, OPTICS, Spectral, Birch

Each classifier was evaluated using accuracy and classification reports, while clustering methods used unsupervised feature embeddings.

### Anomaly Detection

To flag abnormal execution patterns, we implemented:
- Isolation Forest
- One-Class SVM
- Local Outlier Factor
- Elliptic Envelope

These models estimated execution outliers based on deviation from the multivariate normal or density distributions.

### Regression Analysis

To predict Obligations, we trained 16 regressors, including:
- Linear models (Linear, Ridge, Lasso, ElasticNet)
- Robust models (BayesianRidge, Huber, SGD)
- Tree-based (DecisionTree, RandomForest, GradientBoosting, HistGBR, AdaBoost, XGBoost)
- Non-parametric (KNN, SVR, MLP)

Model performance was evaluated using **R²** and **RMSE**, and feature importance was interpreted using:
- Tree-based .feature_importances_
- Permutation importance
- SHAP values

## III. Results

- PCA and t-SNE uncovered latent account structures, validating dimensionality reduction utility.
- Top classification models (GradientBoosting, RandomForest, LogisticRegression) achieved >90% accuracy in identifying high-obligation accounts.
- Anomaly detectors flagged <5% of records as outliers; Isolation Forest showed highest interpretability.
- Regression models explained up to 96.8% of the variance (R²) in obligations, with XGBoost and RandomForest leading.
- Feature importance consistently ranked AnnualAppropriations and BorrowingAuthority as the most influential predictors.
- SHAP and permutation analyses reinforced model trust and transparency.

## IV. Discussion

Our findings confirm the viability of ML techniques for federal budget analysis. Dimensionality reduction aids exploratory understanding, while classification models can segment accounts based on spending intensity. Anomaly detection contributes to audit risk analysis by flagging atypical execution patterns. Regression models are highly effective for predicting obligations, especially when enhanced with ensemble and gradient methods.

Importantly, this study emphasizes interpretability. Through SHAP and permutation analysis, we provide not just prediction, but explanation—an essential requirement in public sector analytics.

Future work will focus on temporal modeling (e.g., forecasting Outlays over time), incorporating account-level metadata (e.g., agency mission), and scaling the framework across fiscal years for comparative analytics.

---

Keywords: federal budget, machine learning, anomaly detection, regression, PCA, XGBoost, SHAP, obligations, appropriations


### Data Processing & Cleaning

In [ ]:
# Load the Excel file
file_path = r'C:\Users\terry\Desktop\Account Balances.xlsx'
xls = pd.ExcelFile( file_path )

# Display all sheet names
sheet_names = xls.sheet_names

# Load all sheets into a dictionary of DataFrames
dfs = { sheet: xls.parse( sheet ) for sheet in sheet_names }

# Display summary of each sheet: name, shape, and column headers
sheet_summaries = {
    sheet: {
        "shape": dfs[ sheet ].shape,
        "columns": dfs[ sheet ].columns.tolist( )
    }
    for sheet in dfs
}

sheet_summaries


In [ ]:
# Load the primary data sheet
df = dfs['Data']

# Select a representative subset of columns for preprocessing and modeling
selected_columns = [
    'AnnualAppropriations', 'BorrowingAuthority', 'ContractAuthority',
    'OffsettingReceipts', 'Obligations', 'Recoveries', 'UnobligatedBalance', 'Outlays'
]
df_numeric = df[selected_columns]

# Drop rows with all NaNs in the selected columns
df_numeric = df_numeric.dropna(how='all')

# Prepare imputers
simple_imputer = SimpleImputer(strategy='mean')
knn_imputer = KNNImputer(n_neighbors=5)

# Prepare scalers
standard_scaler = StandardScaler()
minmax_scaler = MinMaxScaler()

# Impute missing values
df_simple_imputed = pd.DataFrame(simple_imputer.fit_transform(df_numeric), columns=df_numeric.columns)
df_knn_imputed = pd.DataFrame(knn_imputer.fit_transform(df_numeric), columns=df_numeric.columns)

# Scale the data
df_standard_scaled = pd.DataFrame(standard_scaler.fit_transform(df_simple_imputed), columns=df_numeric.columns)
df_minmax_scaled = pd.DataFrame(minmax_scaler.fit_transform(df_simple_imputed), columns=df_numeric.columns)

# PCA on standard scaled data
pca = PCA(n_components=2)
pca_result = pca.fit_transform(df_standard_scaled)

# Truncated SVD on min-max scaled data
svd = TruncatedSVD(n_components=2)
svd_result = svd.fit_transform(df_minmax_scaled)

# Create plots
# 1. PCA Scatter Plot
plt.figure(figsize=(8, 6))
sns.scatterplot(x=pca_result[:, 0], y=pca_result[:, 1])
plt.title("PCA - Standard Scaled Data")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.grid(True)
plt.show()

# 2. Truncated SVD Scatter Plot
plt.figure(figsize=(8, 6))
sns.scatterplot(x=svd_result[:, 0], y=svd_result[:, 1])
plt.title("Truncated SVD - MinMax Scaled Data")
plt.xlabel("Component 1")
plt.ylabel("Component 2")
plt.grid(True)
plt.show()

# 3. Correlation Heatmap (Standard Scaled Data)
plt.figure(figsize=(10, 8))
sns.heatmap(df_standard_scaled.corr(), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Heatmap (Standard Scaled)")
plt.show()

# 4. Histogram of Annual Appropriations (MinMax Scaled)
plt.figure(figsize=(8, 6))
sns.histplot(df_minmax_scaled['AnnualAppropriations'], kde=True, bins=30)
plt.title("Distribution of Annual Appropriations (MinMax Scaled)")
plt.xlabel("Annual Appropriations (Normalized)")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()


###  Exploratory Analysis

In [ ]:
# Step 1: Prepare data
data = df_simple_imputed.copy()
X = data[['AnnualAppropriations', 'BorrowingAuthority', 'ContractAuthority']]
y = data['Obligations']

# Step 2: Z-score normalization
z_scores = pd.DataFrame(zscore(data), columns=data.columns)

# Step 3: T-test (high vs low appropriations)
threshold = data['AnnualAppropriations'].median()
group1 = data[data['AnnualAppropriations'] > threshold]['Obligations']
group2 = data[data['AnnualAppropriations'] <= threshold]['Obligations']
t_stat, t_pval = ttest_ind(group1, group2, nan_policy='omit')

# Step 4: One-way ANOVA on Obligations across Appropriation quartiles
quartiles = pd.qcut(data['AnnualAppropriations'], 4, labels=False, duplicates='drop')
anova_groups = [data.loc[quartiles == i, 'Obligations'] for i in np.unique(quartiles)]
anova_stat, anova_pval = f_oneway(*anova_groups)

# Step 5: Chi-square test using binned values
binned1 = pd.qcut(data['AnnualAppropriations'], 4, labels=False, duplicates='drop')
binned2 = pd.qcut(data['Outlays'], 4, labels=False, duplicates='drop')
contingency_table = pd.crosstab(binned1, binned2)
chi2_stat, chi2_pval, _, _ = chi2_contingency(contingency_table)

# Step 6: Correlation analysis
pearson_corr, pearson_pval = pearsonr(data['AnnualAppropriations'], data['Obligations'])
spearman_corr, spearman_pval = spearmanr(data['AnnualAppropriations'], data['Obligations'])

# Step 7: Regression Model
model = LinearRegression().fit(X, y)
y_pred = model.predict(X)
r2 = model.score(X, y)
n, k = X.shape
adj_r2 = 1 - (1 - r2) * (n - 1) / (n - k - 1)
rss = np.sum((y - y_pred) ** 2)
tss = np.sum((y - y.mean()) ** 2)
explained_var = tss - rss
f_stat = (explained_var / k) / (rss / (n - k - 1))

# Step 8: Coefficient p-values with statsmodels
X_const = sm.add_constant(X)
ols_model = sm.OLS(y, X_const).fit()
coeff_pvals = ols_model.pvalues

# Step 9: Pearson correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(data.corr(), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Pearson Correlation Heatmap")
plt.show()

# Step 10: Summary Output
results_summary = {
    "T-test p-value": t_pval,
    "ANOVA p-value": anova_pval,
    "Chi-square p-value": chi2_pval,
    "Pearson correlation": pearson_corr,
    "Spearman correlation": spearman_corr,
    "R²": r2,
    "Adjusted R²": adj_r2,
    "F-statistic": f_stat,
    "Coefficient p-values": coeff_pvals.to_dict()
}

results_summary


### Clustering & Classification

In [ ]:
# STEP 1: Preprocess Data
df = df_simple_imputed.copy()
df['Target'] = (df['Obligations'] > df['Obligations'].median()).astype(int)

features = ['AnnualAppropriations', 'BorrowingAuthority', 'ContractAuthority', 'OffsettingReceipts']
X = df[features]
y = df['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_full_scaled = scaler.fit_transform(X)  # For clustering


# STEP 2: Define Models

classification_models = {
    "LogisticRegression": LogisticRegression(max_iter=1000),
    "RidgeClassifier": RidgeClassifier(),
    "SGDClassifier": SGDClassifier(max_iter=1000, tol=1e-3),
    "Perceptron": Perceptron(),
    "KNeighborsClassifier": KNeighborsClassifier(),
    "DecisionTreeClassifier": DecisionTreeClassifier(),
    "RandomForestClassifier": RandomForestClassifier(),
    "GradientBoostingClassifier": GradientBoostingClassifier(),
    "HistGradientBoostingClassifier": HistGradientBoostingClassifier(),
    "AdaBoostClassifier": AdaBoostClassifier(),
    "BaggingClassifier": BaggingClassifier(),
    "XGBClassifier": XGBClassifier(use_label_encoder=False, eval_metric='logloss', verbosity=0),
    "SVC": SVC(),
    "GaussianNB": GaussianNB(),
    "BernoulliNB": BernoulliNB(),
    "MLPClassifier": MLPClassifier(max_iter=500),
    "QuadraticDiscriminantAnalysis": QuadraticDiscriminantAnalysis(),
    "LinearDiscriminantAnalysis": LinearDiscriminantAnalysis()
}

clustering_models = {
    "KMeans": KMeans(n_clusters=2, random_state=42),
    "MiniBatchKMeans": MiniBatchKMeans(n_clusters=2, random_state=42),
    "DBSCAN": DBSCAN(eps=0.5, min_samples=5),
    "OPTICS": OPTICS(min_samples=10),
    "AgglomerativeClustering": AgglomerativeClustering(n_clusters=2),
    "Birch": Birch(n_clusters=2),
    "SpectralClustering": SpectralClustering(n_clusters=2, assign_labels='discretize', random_state=42)
}

# STEP 3: Run Classification
classification_results = {}

for name, model in classification_models.items():
    try:
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        accuracy = accuracy_score(y_test, y_pred)
        classification_results[name] = {
            "Accuracy": accuracy,
            "Report": classification_report(y_test, y_pred, output_dict=True)
        }
    except Exception as e:
        classification_results[name] = {"Error": str(e)}

# STEP 4: Run Clustering

clustering_results = {}

for name, model in clustering_models.items():
    try:
        labels = model.fit_predict(X_full_scaled)
        clustering_results[name] = {
            "Labels": labels[:10].tolist(),  # show first 10 to summarize
            "n_clusters": len(np.unique(labels))
        }
    except Exception as e:
        clustering_results[name] = {"Error": str(e)}

# STEP 5: Summaries
classification_summary = {
    name: {"Accuracy": res.get("Accuracy", None)}
    for name, res in classification_results.items()
}

clustering_summary = {
    name: {
        "n_clusters": res["n_clusters"] if "n_clusters" in res else None,
        "SampleLabels": res["Labels"] if "Labels" in res else None,
        "Error": res.get("Error", None)
    }
    for name, res in clustering_results.items()
}

# Output (for notebook use)
print("Classification Summary:")
print(pd.DataFrame(classification_summary).T.sort_values(by="Accuracy", ascending=False))

print("\nClustering Summary:")
print(pd.DataFrame(clustering_summary).T)


### Anomaly & Outlier Detection

In [ ]:

# Subset and scale the original imputed data
anomaly_features = ['AnnualAppropriations', 'BorrowingAuthority', 'ContractAuthority', 'OffsettingReceipts']
X_anomaly = df_simple_imputed[anomaly_features]
X_anomaly_scaled = StandardScaler().fit_transform(X_anomaly)

# Dictionary of anomaly detection models
anomaly_models = {
    "IsolationForest": IsolationForest(contamination=0.05, random_state=42),
    "OneClassSVM": OneClassSVM(nu=0.05, kernel="rbf", gamma='scale'),
    "LocalOutlierFactor": LocalOutlierFactor(n_neighbors=20, contamination=0.05, novelty=True),
    "EllipticEnvelope": EllipticEnvelope(contamination=0.05, random_state=42)
}

# Fit and predict anomaly scores
anomaly_results = {}

for name, model in anomaly_models.items():
    try:
        model.fit(X_anomaly_scaled)
        labels = model.predict(X_anomaly_scaled)  # -1 for outliers, 1 for inliers
        n_outliers = (labels == -1).sum()
        anomaly_results[name] = {
            "OutliersDetected": int(n_outliers),
            "InliersDetected": int((labels == 1).sum())
        }
    except Exception as e:
        anomaly_results[name] = {"Error": str(e)}

anomaly_results


### Dimensionality Reduction

In [ ]:
# 1. Load and scale data
df = df_simple_imputed.copy()
features = ['AnnualAppropriations', 'BorrowingAuthority', 'ContractAuthority', 'OffsettingReceipts']
X = df[features]
X_scaled = StandardScaler().fit_transform(X)

# 2. Apply Dimensionality Reduction
reduction_methods = {
    "PCA": PCA(n_components=2),
    "IncrementalPCA": IncrementalPCA(n_components=2, batch_size=200),
    "TruncatedSVD": TruncatedSVD(n_components=2),
    "FactorAnalysis": FactorAnalysis(n_components=2),
    "Isomap": Isomap(n_components=2, n_neighbors=10),
    "TSNE": TSNE(n_components=2, perplexity=30, random_state=42)
}

reduced_data = {}

for name, reducer in reduction_methods.items():
    try:
        X_red = reducer.fit_transform(X_scaled)
        reduced_data[name] = X_red

        # Plot each result
        plt.figure(figsize=(6, 5))
        plt.scatter(X_red[:, 0], X_red[:, 1], alpha=0.5)
        plt.title(f"{name} - 2D Projection")
        plt.xlabel("Component 1")
        plt.ylabel("Component 2")
        plt.grid(True)
        plt.show()

    except Exception as e:
        print(f"{name} failed: {e}")


### Regression Analysis

In [ ]:
# --- STEP 1: Load & Preprocess Data ---
df = df_simple_imputed.copy()
X = df[['AnnualAppropriations', 'BorrowingAuthority', 'ContractAuthority', 'OffsettingReceipts']]
y = df['Obligations']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- STEP 2: Define Regression Models ---
regressors = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "ElasticNet": ElasticNet(),
    "BayesianRidge": BayesianRidge(),
    "HuberRegressor": HuberRegressor(),
    "SGDRegressor": SGDRegressor(max_iter=1000, tol=1e-3),
    "DecisionTreeRegressor": DecisionTreeRegressor(),
    "RandomForestRegressor": RandomForestRegressor(),
    "GradientBoostingRegressor": GradientBoostingRegressor(),
    "HistGradientBoostingRegressor": HistGradientBoostingRegressor(),
    "AdaBoostRegressor": AdaBoostRegressor(),
    "XGBRegressor": XGBRegressor(verbosity=0, eval_metric='rmse'),
    "KNeighborsRegressor": KNeighborsRegressor(),
    "SVR": SVR(),
    "MLPRegressor": MLPRegressor(max_iter=500)
}

# --- STEP 3: Train & Evaluate Models ---
results = {}
trained_models = {}

for name, model in regressors.items():
    try:
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        r2 = r2_score(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        results[name] = {"R²": round(r2, 5), "RMSE": round(rmse, 2)}
        trained_models[name] = model
    except Exception as e:
        results[name] = {"Error": str(e)}

# --- STEP 4: Print Summary ---
print("\n📊 Regression Summary (R² sorted):")
summary_df = pd.DataFrame(results).T.sort_values(by="R²", ascending=False)
print(summary_df)

# --- STEP 5: Tree-Based Feature Importances ---
tree_models = ["RandomForestRegressor", "GradientBoostingRegressor",
               "HistGradientBoostingRegressor", "AdaBoostRegressor", "XGBRegressor"]

for name in tree_models:
    if name in trained_models and hasattr(trained_models[name], "feature_importances_"):
        importances = trained_models[name].feature_importances_
        plt.figure(figsize=(6, 4))
        plt.barh(X.columns, importances)
        plt.title(f"{name} - Tree-Based Feature Importances")
        plt.xlabel("Importance")
        plt.gca().invert_yaxis()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

# --- STEP 6: Permutation Importance (Model Agnostic) ---
baseline_model = trained_models["RandomForestRegressor"]
perm = permutation_importance(baseline_model, X_test_scaled, y_test, n_repeats=10, random_state=42)

perm_sorted_idx = perm.importances_mean.argsort()
plt.figure(figsize=(6, 4))
plt.barh(np.array(X.columns)[perm_sorted_idx], perm.importances_mean[perm_sorted_idx])
plt.title("Permutation Importance - RandomForest")
plt.xlabel("Mean Importance")
plt.grid(True)
plt.tight_layout()
plt.show()

# --- STEP 7: SHAP Values (Model Agnostic) ---
shap_model = trained_models["GradientBoostingRegressor"]
explainer = shap.Explainer(shap_model.predict, X_test_scaled)
shap_values = explainer(X_test_scaled)

# Summary Plot (SHAP)
shap.plots.beeswarm(shap_values, max_display=10)

